In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
from collections import Counter

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [230]:
#loading data
headlines = pd.read_csv('FiQA_train_ABSA_financial_headlines.tsv',delimiter='\t',header=0)
posts = pd.read_csv('FiQA_train_ABSA_financial_posts.tsv',delimiter='\t',header=0)
posts = posts.drop(['Unnamed: 0', 'id'],axis=1)
headlines = headlines.drop(['Unnamed: 0', 'id'],axis=1)
posts_text = posts.iloc[:,:1]
posts_score = posts.iloc[:,1:]
headlines_text = headlines.iloc[:,:1]
headlines_score = headlines.iloc[:,1:]
text_frames = [posts_text,headlines_text]
score_frames = [posts_score,headlines_score]
text = pd.concat(text_frames)
score = pd.concat(score_frames)
# text = pd.read_csv('reviews.txt', header=None)
# score = pd.read_csv('labels.txt', header=None)

In [231]:
#counting word frequency to create vocabulary
counter_ = Counter()
for txt in text.values:
    for word in txt[0].split(" "):
        counter_[word]+=1

In [232]:
print(len(counter_))

11384


In [237]:
#keeping only 1000 most commmon words, frequency of 1000th word can be used to see if we need ot increase or decrease this number
vocabulary = sorted(counter_,key=counter_.get,reverse = True)[:1000]

In [238]:
counter_[vocabulary[-1]]

9

In [239]:
#word to index, will need this mapping to create word vector 
word_to_index = dict()
for index,word in enumerate(vocabulary):
    word_to_index[word] = index

In [240]:
def get_word_vector(data):
    vector = np.zeros(len(vocabulary), dtype=np.int_)
    for word in data.split(' '):
        index = word_to_index.get(word, None)
        if index != None:
            vector[index]+=1
    return vector

In [241]:
text_vectors = np.zeros((len(text),len(vocabulary)),dtype=np.int_)
for index,(_,data) in enumerate(text.iterrows()):
    text_vectors[index] = get_word_vector(data[0])

In [242]:
text_vectors[:5]

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [245]:
#converting scores into positive and negative classes
Y = (score>=0).astype(np.int_)
Y['sentiment score'].value_counts()

1    2407
0    1382
Name: sentiment score, dtype: int64

In [249]:
shuffle = np.arange(len(score))
np.random.shuffle(shuffle)
test_train_split = 0.9
train_split, test_split = shuffle[:int(len(score)*test_train_split)], shuffle[int(len(score)*test_train_split):]
trainX, trainY = text_vectors[train_split,:], to_categorical(Y.values[train_split].ravel(), 2)
testX, testY = text_vectors[test_split,:], to_categorical(Y.values[test_split].ravel(), 2)

In [250]:
trainX = tflearn.data_utils.pad_sequences(trainX, maxlen=100, value=0.)
testX = tflearn.data_utils.pad_sequences(testX, maxlen=100, value=0.)
trainY

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [251]:
# Network building
# tf.reset_default_graph()
# net = tflearn.input_data([None, 100])
# net = tflearn.embedding(net, input_dim=len(vocabulary), output_dim=128)
# net = tflearn.lstm(net, 128, dropout=0.8)
# net = tflearn.fully_connected(net, 2, activation='softmax')
# net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
#                          loss='categorical_crossentropy')

tf.reset_default_graph()
net = tflearn.input_data([None, 100])                          # Input
net = tflearn.fully_connected(net, 200, activation='ReLU')      # Hidden
net = tflearn.fully_connected(net, 25, activation='ReLU')
net = tflearn.fully_connected(net, 2, activation='softmax')   # Output
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')


In [252]:
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY,n_epoch=100,validation_set=0.1, show_metric=True,batch_size=32)

Training Step: 9599  | total loss: 0.23581 | time: 0.381s
| SGD | epoch: 100 | loss: 0.23581 - acc: 0.8935 -- iter: 3040/3069
Training Step: 9600  | total loss: 0.23032 | time: 1.388s
| SGD | epoch: 100 | loss: 0.23032 - acc: 0.8947 | val_loss: 0.99878 - val_acc: 0.6686 -- iter: 3069/3069
--


In [255]:
# (np.array(model.predict(testX))[:,0]>=0.5).astype(np.int_)

predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Test accuracy: ", test_accuracy)
predictions

Test accuracy:  0.68073878628


array([1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0,